<a href="https://colab.research.google.com/github/jaedyxn/ait23-project/blob/main/datasplit.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv("cleaned.csv")

In [ ]:
df.head()

,Unnamed: 0,CONS_NO,FLAG,1/1/2014,1/10/2014,1/11/2014,1/12/2014,1/13/2014,1/14/2014,1/15/2014,...,9/28/2016,9/29/2016,9/3/2016,9/30/2016,9/4/2016,9/5/2016,9/6/2016,9/7/2016,9/8/2016,9/9/2016
0,0,0387DD8A07E07FDA6271170F86AD9151,1,13.876554,13.876554,13.876554,13.876554,13.876554,13.876554,13.876554,...,10.12,9.96,16.92,7.60,27.22,18.05,26.47,18.75,17.84,14.92
1,1,EDFC78B07BA2908B3395C4EB2304665E,1,2.900000,3.420000,3.810000,4.580000,3.560000,4.250000,3.860000,...,17.77,10.37,15.32,13.51,12.23,14.68,16.35,18.14,18.41,17.31
2,2,34C1954AA3703C4F8BD8EAEA7C4B7B83,1,0.110000,0.530000,0.450000,0.510000,1.320000,0.710000,0.120000,...,4.33,2.46,1.23,3.19,1.17,1.36,0.56,1.02,1.96,1.83
3,3,768309B0EB11FD436CEE5ABFB84F4C0C,1,0.910000,0.860000,1.100000,0.660000,5.820000,3.170000,1.180000,...,2.36,2.82,2.28,3.70,3.07,4.61,3.13,3.10,3.74,3.13
4,4,516954F5FF177CE314656D727FCC66A5,1,11.020000,8.240000,7.940000,7.920000,8.310000,7.390000,8.270000,...,51.36,52.39,113.35,60.89,115.62,108.69,87.88,86.36,81.51,81.75


In [ ]:
df.shape

(31188, 1037)

In [ ]:
first0 = 0
last1 = 0
for index, row in df.iterrows():
    if row['FLAG'] == 0 and first0 == 0:
        first0 = index
    if row['FLAG'] == 1:
        last1 = index
print("last 1 flag: " + str(last1))
print("first 0 flag: " + str(first0))




first 0 flag: 2409
last 1 flag: 2408


In [ ]:
Y = df.filter(['0', 'FLAG'], axis = 1)

In [ ]:
Y.shape

(31188, 1)

In [ ]:
X = df.drop(columns = ['FLAG'])

In [ ]:
X.shape

(31188, 1036)

In [ ]:
valid_split = 0.2
test_split  = 0.1

v_point_1 = int(last1*(1-valid_split-test_split))
t_point_1 = int(last1*(1-test_split))


X_train = X[:v_point_1]
Y_train = Y[:v_point_1]
X_valid = X[v_point_1:t_point_1]
Y_valid = Y[v_point_1:t_point_1]
X_test  = X[t_point_1:first0]
Y_test  = Y[t_point_1:first0]

v_point_0 = int((len(X)-last1)*(1-valid_split-test_split))
t_point_0 = int((len(X)-last1)*(1-test_split))

X_train = X_train.append(X[first0:v_point_0])
Y_train = Y_train.append(Y[first0:v_point_0])
X_valid = X_valid.append(X[v_point_0:t_point_0])
Y_valid = Y_valid.append(Y[v_point_0:t_point_0])
X_test  = X_test.append(X[t_point_0:])
Y_test  = Y_test.append(Y[t_point_0:])

print(len(X_train) + len(X_test) + len(X_valid))


31188


# Model Definition

In [ ]:
model = Sequential()
model.add(LSTM(128, input_shape=(X.shape[-2], X.shape[-1])))
model.add(Dense(128))
model.add(Activation('softmax'))

optimizer = RMSprop(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

# Model Training

In [ ]:
es = EarlyStopping(patience=5, restore_best_weights=True, monitor="val_accuracy")

model.fit(X_train, y_train, 
          epochs=50, batch_size=128, 
          validation_data=(X_val, y_val), # validation
          callbacks=[es],
          verbose=1)

# Evaluation on test data

In [ ]:
eval = model.evaluate(X_test, y_test)
print("Test loss:",eval[0])
print("Test accuracy:",eval[1])

# Confusion matrix

In [ ]:
preds = model.predict(X_test)
conf=confusion_matrix(np.argmax(y_test,1),np.argmax(preds,1))
sns.heatmap(conf, annot=True, fmt='d', vmax=100)